In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [2]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *
qvm = api.QVMConnection()

In [3]:
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize

In [4]:
from pyquil.paulis import sX, sY, sZ, ID

We are going to try to use VQE to calculate the minimum energy fo LiH. We will use VQE in Pyquil/Grove and use the Nelder-Mead Algorithm to start.

The first thing we are doing is representing the hamiltonian of this system in Pyquil.

In [5]:
hamiltonian = -0.096022*sZ(0) + -0.206128*sZ(0)*sZ(1) + 0.364746*sZ(1) + 0.096022*sZ(2) + -0.206128*sZ(2)*sZ(3) + -0.364746*sZ(3) + -0.145438*sZ(0)*sZ(2) + 0.056040*sZ(0)*sZ(2)*sZ(3) + 0.110811*sZ(0)*sZ(3) + -0.056040*sZ(0)*sZ(1)*sZ(2) + 0.080334*sZ(0)*sZ(1)*sZ(2)*sZ(3) + 0.063673*sZ(0)*sZ(1)*sZ(3) + 0.110811*sZ(1)*sZ(2) + -0.06373*sZ(1)*sZ(2)*sZ(3) + -0.095216*sZ(1)*sZ(3)
+ -0.012585*sX(0)*sZ(1) + 0.012585*sX(0) + 0.012585*sX(2)*sZ(3) + 0.012585*sX(2) + -0.002667*sX(0)*sZ(1)*sX(2)*sZ(3) + -0.002667*sX(0)*sZ(1)*sX(2) + 0.002667*sX(0)*sX(2)*sZ(3) + 0.002667*sX(0)*sX(2) + 0.007265*sX(0)*sZ(1)*sZ(3) + -0.007265*sX(0)*sZ(3) + 0.007265*sZ(1)*sX(2)*sZ(3) + 0.007265*sZ(1)*sX(2)
+ -0.029640*sX(0)*sX(1) + 0.002792*sX(1) + -0.029640*sX(2)*sX(3) + 0.002792*sX(3) + -0.008195*sX(0)*sX(2)*sX(3) + -0.001271*sX(0)*sX(3) + -0.008195*sX(0)*sX(1)*sX(2) + 0.028926*sX(0)*sX(1)*sX(2)*sX(3) + 0.007499*sX(0)*sX(1)*sX(3) + -0.001271*sX(1)*sX(2) + 0.007499*sX(1)*sX(2)*sX(3) + 0.009327*sX(1)*sX(3)
+ 0.029640*sY(0)*sY(1) + 0.029640*sY(2)*sY(3) + 0.028926*sY(0)*sY(1)*sY(2)*sY(3)
+ 0.002792*sZ(0)*sX(1) + -0.002792*sZ(2)*sX(3) + -0.016781*sZ(0)*sZ(2)*sX(3) + 0.016781*sZ(0)*sX(3) + -0.016781*sZ(0)*sX(1)*sZ(2) + -0.016781*sX(1)*sZ(2) + -0.009327*sZ(0)*sX(1)*sZ(2)*sX(3) + 0.009327*sZ(0)*sX(1)*sX(3) + -0.009327*sX(1)*sZ(2)*sX(3)
+ -0.011962*sZ(0)*sX(2)*sZ(3) + -0.011962*sZ(0)*sX(2) + 0.000247*sZ(0)*sZ(1)*sX(2)*sZ(3) + 0.000247*sZ(0)*sZ(1)*sX(2)
+ 0.039155*sZ(0)*sX(2)*sX(3) + -0.002895*sZ(0)*sZ(1)*sX(2)*sX(3) + -0.009769*sZ(0)*sZ(1)*sX(3) + -0.024280*sZ(1)*sX(2)*sX(3) + -0.008025*sZ(1)*sX(3)
+ -0.039155*sZ(0)*sY(2)*sY(3) + 0.002895*sZ(0)*sZ(1)*sY(2)*sY(3) + 0.024280*sZ(1)*sY(2)*sY(3)
+ -0.011962*sX(0)*sZ(1)*sZ(2) + 0.011962*sX(0)*sZ(2) + -0.000247*sX(0)*sZ(1)*sZ(2)*sZ(3) + 0.000247*sX(0)*sZ(2)*sZ(3)
+ 0.008195*sX(0)*sZ(1)*sX(2)*sX(3) + 0.0001271*sX(0)*sZ(1)*sX(3) + -0.007499*sX(0)*sX(1)*sZ(2)*sX(3)
+ -0.008195*sX(0)*sZ(1)*sY(2)*sY(3) + 0.008195*sX(0)*sY(2)*sY(3) + 0.007499*sY(0)*sY(1)*sZ(2)*sX(3)
+  -0.001271*sX(0)*sZ(1)*sZ(2)*sX(3) + 0.001271*sX(0)*sZ(2)*sX(3) + 0.008025*sZ(1)*sZ(2)*sX(3) + 0.009769*sZ(0)*sZ(1)*sZ(2)*sX(3)
+ -0.039155*sX(0)*sX(1)*sZ(2) + -0.002895*sX(0)*sX(1)*sZ(2)*sZ(3) + 0.024280*sX(0)*sX(1)*sZ(3) + -0.009769*sX(1)*sZ(2)*sZ(3) + 0.008025*sX(1)*sZ(3) + -0.001271*sZ(0)*sX(1)*sX(2)*sZ(3) + -0.001271*sZ(0)*sX(1)*sX(2) + 0.008025*sZ(0)*sX(1)*sZ(3)
+ 0.039155*sY(0)*sY(1)*sZ(2) + 0.002895*sY(0)*sY(1)*sZ(2)*sZ(3) + -0.024280*sY(0)*sY(1)*sZ(3) + 0.007499*sZ(0)*sX(1)*sX(2)*sX(3)
+ -0.008195*sX(0)*sX(1)*sX(2)*sZ(3) + -0.001271*sX(1)*sX(2)*sZ(3) + -0.007499*sZ(0)*sX(1)*sY(2)*sY(3)
+ 0.008195*sY(0)*sY(1)*sX(2)*sZ(3) + 0.008195*sY(0)*sY(1)*sX(2) + -0.009769*sZ(0)*sX(1)*sZ(2)*sZ(3)
+ -0.028926*sX(0)*sX(1)*sY(2)*sY(3) +  -0.007499*sX(1)*sY(2)*sY(3)  + -0.028926*sY(0)*sY(1)*sX(2)*sX(3) + -0.007499*sY(0)*sY(1)*sX(3)

We will now create a small ansatz function. This provides the gates that give us our states. We will then run our VQE using these gates to find the minimum energy.

In [6]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0), RY(params[1], 0), RZ(params[2],0),
                   RX(params[3], 1), RY(params[4], 1), RZ(params[5],1),
                   RX(params[6], 2), RY(params[7], 2), RZ(params[8],2),
                   RX(params[9], 3), RY(params[10], 3), RZ(params[11],3)
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14158201e+00,   7.80035253e-06,   4.49390667e+00,
        -8.79683447e-07,   3.14158407e+00,   3.14369079e+00,
        -3.76259573e-06,   2.66922549e-05,  -1.22785945e+00,
         8.96356157e-06,   3.93712138e-06,   1.96039179e+00]), 'fun': -0.83501499994339268}


Groundstate energy is approximately -0.835015, which agrees with what we saw using numpy.

Now let's try using the Powell minimization algorithm.

In [7]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0), RY(params[1], 0), RZ(params[2],0),
                   RX(params[3], 1), RY(params[4], 1), RZ(params[5],1),
                   RX(params[6], 2), RY(params[7], 2), RZ(params[8],2),
                   RX(params[9], 3), RY(params[10], 3), RZ(params[11],3)
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'Powell'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14159263e+00,   1.70107798e-08,   4.62731123e+00,
         3.14159264e+00,   1.32820280e-08,   2.96108936e+00,
         6.19718004e-08,   3.59081157e-08,   2.43573981e+00,
         2.95729374e-08,   1.32035854e-09,   5.06501714e+00]), 'fun': array(-0.8350150000000002)}


Now let's try the CG algorithm.

In [8]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0), RY(params[1], 0), RZ(params[2],0),
                   RX(params[3], 1), RY(params[4], 1), RZ(params[5],1),
                   RX(params[6], 2), RY(params[7], 2), RZ(params[8],2),
                   RX(params[9], 3), RY(params[10], 3), RZ(params[11],3)
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'CG'})
initial_angles_HXYZ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([ -3.23741747e-05,  -3.23822102e-05,   9.99999549e-01,
         1.57075317e+00,   1.57083943e+00,   9.99999746e-01,
        -8.47096160e-08,  -1.08065279e-07,   9.99999747e-01,
        -2.97716039e-06,  -2.96565961e-06,   9.99999902e-01]), 'fun': -0.5494610002874158}


Now we will run VQE using three RX gates and again using only two RX gates, all for the Nelder-Mead algorithm, and see what we get.

In [9]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1), 
                   RX(params[2], 2), 
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([  3.14162753e+00,   3.14157891e+00,  -8.85630990e-06]), 'fun': -0.83501499993072792}


In [10]:
def smallish_ansatz_HXYZ(params):
    return Program(RX(params[0], 0),
                   RX(params[1], 1),
)

vqe_inst = VQE(minimizer=minimize,
               minimizer_kwargs={'method': 'nelder-mead'})
initial_angles_HXYZ = [1.0, 1.0]
result = vqe_inst.vqe_run(smallish_ansatz_HXYZ, hamiltonian, initial_angles_HXYZ, None, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([-3.14162319,  3.14157863]), 'fun': -0.83501499993695971}
